# Getting Started with ``spore``

``spore`` is a set of modules/classes/functions to self-consistently create low-frequency radio interferometric EoR foreground models and analyse them in different ways.
It centres around 3 basic tasks, each implemented as its own sub-package, which can interact to some degree: 
1. ``model``: define individual foreground model components analytically, and combine them analytically.
2. ``mock``: combine different ``model``s to produce a sampled realisation of the sky.
3. ``measure``: given a realisation of the sky (either a result from a ``mock`` or from some external simulation), make measurements of the power spectrum on it.

``spore`` also contains a sub-package dedicated to visualising these results.

In this notebook we'll show some simple examples of how to use the above three sub-packages, focusing on creating ``model``s. First, we'll do a few imports to set us up.

In [ ]:
import numpy as np
from spore.mock.foregrounds import PointSourceForegrounds, PointSourceForegroundsDirect

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Model

At this time, ``spore`` defines 4 kinds of models (we'll call them component models): beam models, source counts, spatial distributions, and spectral index distributions. They are imported as follows:

In [ ]:
from spore.model import beam, source_counts, spatial_dist, spectral_index

Each module comprises a base component class, along with some number of specific models for that component. The idea behind this layout is that you can define your own specific models for that component in your own code, by subclassing the base component class and following its API. Let's have a brief look at each component.

### Beam Models

The base component class for beam models is ``Beam``. It's input API is such that 1 necessary parameter is present: ``nu0`` which defines the reference frequency for the beam parameters.

The main output API that the subclass must implement is ``beam``, which takes a vector of ``l`` (and optionally ``m``) and evaluates beam attenuation at those co-ordinates, at frequency ``f0`` which defines the frequency (as a ratio to ``nu0``) at which the beam quantities are to be evaluated. This latter can be an array. If so, the returned array has an extra dimension (which can get quite large very quickly).

The obvious simple model is a Circular Gaussian beam, and this is implemented within the class. To call it, use

In [ ]:
pl_source_counts.dnds(0.5)

In addition, one can evaluate any moment of the source counts (defined as $\mu_n = \int S^n \frac{dN}{dS} dS$):

In [ ]:
pl_source_counts.mu(1)

Finally, one can sample the source counts (at $\nu_0$):

In [ ]:
fluxes = pl_source_counts.sample_source_counts(10000)

plt.hist(np.log10(fluxes.value),bins=50)
plt.yscale('log')
plt.xlabel("Log10 Flux Density")

### Spectral Index

At this point, there are only a couple of spectral index distribution models. Note that these all assume that a source has a power-law SED within the bandwidth of interest. Perhaps later versions will generalise to more complicated SED's. The most common/easy spectral index distribution is the universal one (i.e. every source has the same spectral index):

In [ ]:
spec_index_dist = spectral_index.UniversalDist(0.8)

This object has a method to sample this distribution:

In [ ]:
spec_index_dist.sample(n=10)

### Spatial Distributions

There are two fundamentally different kinds of spatial distribution -- those that have shot noise from the discreteness of sources, and those that do not. All of these are contained in the ``spatial_dist`` module, and are subclassed from the ``SpatialDistribution`` base component class.

We'll use an example of both here:

In [ ]:
pure_sky = spatial_dist.PureClustering_FlatSky(f0 = 1., # Frequencies (as ratios to ref. freq.) to evaluate sky at.
                                               sky_size = 2.0,             # Size of sky in l,m co-ordinates
                                               ncells = 256,              # Number of grid cells along a side 
                                               seed = 1234,                # Optional seed to enforce the same realisation on every run
                                               power_spectrum = lambda u : (u/0.1)**-2.0   # A function defining an isotropic power spectrum of sources as a function of scale u.
                                              )

# Define poisson sky with all the same parameters
poisson_sky = spatial_dist.PoissonClustering_FlatSky(
                                               f0 = 1.,
                                               sky_size = 2.0,             
                                               ncells = 256,              
                                               seed = 1234,                
                                               power_spectrum = lambda u : (u/0.1)**-2.0
                                              )


To create a uniformly distributed sky, one just needs to not pass the ``power_spectrum`` argument

The most relevant method of these classes is ``sky``. It gives a grid of sky brightnesses, at all frequencies. One thing to keep in mind is the "pure" clustering classes have a different signature for this method. They require only a mean brightness (for all frequencies), whereas "poisson" classes require an array of source positions, fluxes and spectral indices. We'll show both here:

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,7))

# The pure clustering sky call.
puresky = pure_sky.sky(sbar=[10.])

# For poisson clustering, need a sample of positions. Poisson classes have a method to do this:
source_pos = poisson_sky.source_positions(nbar=100000.)
npos = len(source_pos[0])
fluxes = np.random.uniform(size=npos)
specs = 0.8 * np.ones(npos)

poissonsky = poisson_sky.sky(source_pos, fluxes, specs)

# Plot it up
ax[0].imshow(puresky[0,:,:].T, extent=(pure_sky.lgrid[0].min().value,pure_sky.lgrid[0].max().value,pure_sky.lgrid[0].min().value,pure_sky.lgrid[0].max().value), origin='lower')
ax[1].imshow(poissonsky[0,:,:].T, extent=(pure_sky.lgrid[0].min().value,pure_sky.lgrid[0].max().value,pure_sky.lgrid[0].min().value,pure_sky.lgrid[0].max().value), origin='lower')

Of course, we can also plot say a scatter-plot of the source positions themselves:

In [ ]:
plt.figure(figsize=(7.5,6))
plt.scatter(source_pos[0], source_pos[1], s=1, c=fluxes, alpha = 0.3)
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.colorbar()

From v0.0.3, there are also built-in classes for populating the sky treating solid-angles as equal, rather than $l,m$-coordinates. We'll show an example of this in the next section, but show here how to create them:

In [ ]:
poisson_sky_sph = spatial_dist.PoissonClustering_Spherical(
                                               f0 = 1.,
                                               sky_size = 2.0,             
                                               ncells = 256,              
                                               seed = 1234,                
                                               power_spectrum = lambda l : (l/0.3)**-2.0,
                                               nside = 64,  # Resolution of underlying spherical HEALpix grid
                                               lmax = 1000  # Resolution of power spectrum itself
                                              )

pure_sky_sph = spatial_dist.PureClustering_Spherical(
                                               f0 = 1.,
                                               sky_size = 2.0,             
                                               ncells = 256,              
                                               seed = 1234,                
                                               power_spectrum = lambda l : (l/0.3)**-2.0,
                                               nside = 64,  # Resolution of underlying spherical HEALpix grid
                                               lmax = 1000  # Resolution of power spectrum itself
                                              )


## Mock

In this section, we put the models of the first section together to create a self-consistent mock. The way to do this should seem reasonably obvious by inspecting the spatial distribution setup, however, we wrap all of this in a single class. At this point, the class only deals with point-source foregrounds. To use it:

In [ ]:
gaussbeam = beam.CircularGaussian(nu0=150., D=4.) #Note D is the diameter of the array tile.

One can evaluate attenuation at any co-ordinate:

In [ ]:
gaussbeam.beam(np.array([0]))

In [ ]:
gaussbeam.beam(np.array([0]), np.array([0.2]))

In [ ]:
gaussbeam.beam(np.linspace(0,1,10), f0=np.array([1,1.5]))

### Source Counts Models

The base component class for source counts is ``SourceCounts``. To it, one must pass ``Smin0`` (the faintest sources at ``nu0``), ``Smax0`` (the brightest sources at ``nu0``), ``f0`` and ``spectral_index``. Along with these, each subclass will define its own set of additional parameters. The simplest example is a power-law:

In [ ]:
pl_source_counts = source_counts.PowerLawSourceCounts(Smin0=1e-2, Smax0=1.0, f0=1.0, # Default for spectral_index is 0.8
                                                      alpha=6998.,  # Normalisation of powr=er-law
                                                      beta =1.54   # Slope of power-law
                                                     )

One can evaluate a few simple quantities:

In [ ]:
print(pl_source_counts.total_number_density)
print(pl_source_counts.total_flux_density)
print(pl_source_counts.total_squared_flux_density)

One can evaluate the actual source counts at any flux density (at $\nu_0$):

We pass each of the models we have created in the previous section. All of the models get saved into the class under the names they are passed as (so you can access ``psd_pure.beam_model`` for instance). Some more methods are also available, which tie together the various components. From this, we can plot the sky distribution and other quantities. Note that all of these models inherently provide the visibility gridded into the $(u,v)$ plane on a regular grid, which is defined in turn by the extent of the sky. 

Alternatively, one may use a ``Direct`` foreground model, which is able to directly calculate the visibility over an irregular $(u,v)$ layout:

In [ ]:
u0 = psf_poiss.ugrid_raw0 # This is the u values on a side of the regular grid.
N_u = len(u0)
u0,v0 = np.meshgrid(u0,u0) #Construct a 2D square of values 

psf_poiss_direct = PointSourceForegroundsDirect(u0 = u0.flatten(), v0=v0.flatten(),
                                                beam_model=gaussbeam,
                                                source_counts = pl_source_counts,
                                                spec_index_model = spec_index_dist,
                                                spatial_dist = poisson_sky)

### Sky Distribution

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))

lmin = psf_pure.spatial_dist.lgrid[0].min().value
lmax = psf_pure.spatial_dist.lgrid[0].max().value

ax[0,0].imshow(psf_pure.visible_sky()[0,:,:].T, 
             extent=(lmin,lmax,lmin,lmax), 
             origin='lower')

ax[0,1].imshow(psf_poiss.visible_sky()[0,:,:].T, 
             extent=(lmin,lmax,lmin,lmax), 
             origin='lower')

ax[1,0].imshow(psf_pure_sph.visible_sky()[0,:,:].T, 
             extent=(lmin,lmax,lmin,lmax), 
             origin='lower')

ax[1,1].imshow(psf_poiss_sph.visible_sky()[0,:,:].T, 
             extent=(lmin,lmax,lmin,lmax), 
             origin='lower')

plt.show()

### Visibilities

We can also determine the visibilities over a uniform grid:

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,7))

umin = psf_pure.ugrid_raw0.min().value
umax = psf_pure.ugrid_raw0.max().value

ax[0].imshow(np.real(psf_pure.visibility[0,:,:].value),
             extent=(umin,umax,umin,umax), 
             origin='lower',vmin=-100., vmax = 100.)

ax[1].imshow(np.real(psf_poiss.visibility[0,:,:].value), 
             extent=(umin,umax,umin,umax), 
             origin='lower',vmin=-100., vmax = 100.)

# This takes quite a while because of the size of u0.
ax[2].imshow(np.real(psf_poiss_direct.visibility[0].reshape((N_u,N_u))), 
             extent=(umin,umax,umin,umax), 
             origin='lower',vmin=-100., vmax = 100.)

plt.show()

More usefully, we can determine circularly averaged quantities, such as the circularly averaged square visibility:

In [ ]:
plt.plot(psf_pure.ugrid, psf_pure.visibility_squared_circular[0,0,:],    # Note the first two dimensions refer to frequency -- we just use nu0
         label="Pure") 
plt.plot(psf_poiss.ugrid, psf_poiss.visibility_squared_circular[0,0,:],
         label="Poisson")

plt.legend()
plt.xscale('log')
plt.yscale('log')

You can see the smallest scales have a significant shot-noise contribution.

### Covariance of Visibilities

Because it is such an important quantity, we also provide a function for generating the covariance of visibilities as a function of scale $u$. One can use it like this:

In [ ]:
from spore.mock.utils import visibility_covariance

Basically, to the function we just pass a number of iterations, and then the arguments we would usually pass to the ``PointSourceForegrounds`` constructor. 
One thing to be careful of is *not* to set a seed in the spatial distribution -- otherwise each iteration will have the same realisation!

In [ ]:
psf_pure = PointSourceForegrounds(beam_model=gaussbeam,
                                  source_counts=pl_source_counts,
                                  spec_index_model = spec_index_dist,
                                  spatial_dist = pure_sky)

psf_poiss = PointSourceForegrounds(beam_model=gaussbeam,
                                  source_counts=pl_source_counts,
                                  spec_index_model = spec_index_dist,
                                  spatial_dist = poisson_sky)

psf_pure_sph = PointSourceForegrounds(beam_model=gaussbeam,
                                  source_counts=pl_source_counts,
                                  spec_index_model = spec_index_dist,
                                  spatial_dist = pure_sky_sph)

psf_poiss_sph = PointSourceForegrounds(beam_model=gaussbeam,
                                  source_counts=pl_source_counts,
                                  spec_index_model = spec_index_dist,
                                  spatial_dist = poisson_sky_sph)


### Generate 2D power spectrum

To generate the 2D power spectrum, we create a foreground object that contains a regular array of frequencies, as we'll need those to form the second dimensions of the 2D power spectrum:

In [ ]:
f0 = np.linspace(1,1.6/1.5, 100)    # Gives 100 kHz channels between 150 MHz and 160 MHz

fg = Foregrounds(
    beam_model=beam.CircularGaussian(
        nu0   = 150.,             # The only place the actual reference frequency is defined (in MHz)
        D     = 4.,               # The diameter of a tile (for MWA)
    ),
    galactic_kwargs = dict(
        fluctation_temp=253*0.8,  # Unused
        Aeff = 20.,
        spec_index_model=gal_spec_index,
        spatial_dist = GSM(
            f0       = f0,        # Frequencies (as ratios to ref. freq.) to evaluate sky at.
            sky_size = 2.0,       # Size of sky in l,m co-ordinates
            ncells   = 512,       # Number of grid cells along a side 
            theta0   = 3*np.pi/4.,   # Zenith pointing
            low_res  = True       # Use low-res base maps for speed
        )
    ),
    point_source_kwargs = dict(
        spec_index_model=spec_index_dist,
        spatial_dist = spatial_dist.PoissonClustering_Spherical(
            f0       = f0,
            sky_size = 2.0,       # Must be the same as the Galactic spatial_dist sky_size
            ncells   = 512,       # Must be the same as the Galactic spatial_dist ncells
            seed     = 1234,      # Just so we get the same results every time we run the notebook  
            nside    = 256,       # Resolution of underlying spherical HEALpix grid
            lmax     = 1000,      # Resolution of power spectrum itself
            power_spectrum = lambda l : (l/0.3)**-2.0,
            
        ),
        source_counts = source_counts.PowerLawSourceCounts(
            Smin0    = 1e-2, 
            Smax0    = 1., 
            f0       = f0,
            alpha    = 6998.,     # Normalisation of power-law
            beta     = 1.54       # Slope of power-law
        )
    )
)

We'll first check the simple underlying gridded sky/visibility for all components (we do this at first and last frequencies alternatively to check that all the frequencies "worked"):

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(10,5))

ax[0,0].imshow(fg.sky[0].T, origin='lower')
ax[0,1].imshow(fg.point_source_foregrounds.sky[0].T, origin='lower')
ax[0,2].imshow(fg.galactic_foregrounds.sky[0].T, origin='lower')

ax[1,0].imshow(fg.visible_sky()[-1].T, origin='lower')
ax[1,1].imshow(fg.point_source_foregrounds.visible_sky()[-1].T, origin='lower')
ax[1,2].imshow(fg.galactic_foregrounds.visible_sky()[-1].T, origin='lower')


Check the visibilities:

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,5))

uextent = (fg.ugrid_raw0[0].value, fg.ugrid_raw0[-1].value, fg.ugrid_raw0[0].value, fg.ugrid_raw0[-1].value)

ax[0].imshow(np.abs(fg.visibility[0].T).value, origin='lower', norm=LogNorm(), extent=uextent)
ax[1].imshow(np.abs(fg.point_source_foregrounds.visibility[0].T).value, origin='lower', norm=LogNorm(), extent=uextent)
ax[2].imshow(np.abs(fg.galactic_foregrounds.visibility[0].T).value, origin='lower', norm=LogNorm(), extent=uextent)

Now we can look at the 2D PS of the grid:

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,5), subplot_kw={"xscale":'log', 'yscale':'log'})

ps_2d, kperp, ubins = fg.power_spectrum_2d(natural_units=True, taper=np.blackman)
ax[0].imshow(ps_2d.value, extent=(kperp.min().value, kperp.max().value, fg.kpar.min().value, fg.kpar.max().value), 
             origin='lower', norm=LogNorm())

ps_2d, kperp, ubins = fg.point_source_foregrounds.power_spectrum_2d(natural_units=True, taper=np.blackman)
ax[1].imshow(ps_2d.value, extent=(kperp.min().value, kperp.max().value, fg.kpar.min().value, fg.kpar.max().value), 
             origin='lower', norm=LogNorm())

ps_2d, kperp, ubins = fg.galactic_foregrounds.power_spectrum_2d(natural_units=True, taper=np.blackman)
ax[2].imshow(ps_2d.value, extent=(kperp.min().value, kperp.max().value, fg.kpar.min().value, fg.kpar.max().value), 
             origin='lower', norm=LogNorm())


In [ ]:
plt.plot(ps_2d[0,:])
plt.plot(ps_2d[10,:])
plt.plot(ps_2d[20,:])
plt.plot(ps_2d[30,:])

plt.xscale('log')
plt.yscale('log')

In [ ]:
fg.ugrid_raw0[1]-fg.ugrid_raw0[0]

Let's make a FlatSky Poisson foreground model for comparison

In [ ]:
ps_fg_flat = PointSourceForegrounds(
        beam_model=fg.beam_model,
        spec_index_model=fg.point_source_foregrounds.spec_index_model,
        source_counts = fg.point_source_foregrounds.source_counts,
        spatial_dist = spatial_dist.PoissonClustering_FlatSky(
            f0       = f0,
            sky_size = 2.0,       
            ncells   = 512,       
            seed     = 4321,
            power_spectrum = lambda l : (l/0.3)**-2.0,
        )
)

In [ ]:
from scipy.signal import blackmanharris

ps_2d, kperp, ubins = ps_fg_flat.power_spectrum_2d(natural_units=True, taper=blackmanharris)
plt.imshow(ps_2d.value, extent=(kperp.min().value, kperp.max().value, fg.kpar.min().value, fg.kpar.max().value), 
             origin='lower', norm=LogNorm())
plt.xscale('log')
plt.yscale('log')

In [ ]:
from powerbox.dft import fft

In [ ]:
from spore.measure.ps_2d_from_single_vis import ps_3d_to_ps_2d

In [ ]:
ft3d,eta = fft(fg.visibility.value, L=0.1, a=0, b = 2*np.pi, axes=(0,))
ft,eta = fft(fg.visibility[:,100,50], L=0.1, a=0, b = 2*np.pi)

plt.plot(eta[0],np.abs(ft3d[:,100,50])**2)
plt.plot(eta[0],np.abs(ft)**2)


In [ ]:
plt.imshow(np.abs(ft3d[:,200,:])**2)

Now import the positions of the tiles (for the MWA) and convert them to physical baselines:

In [ ]:
ps_2d, ubins = ps_3d_to_ps_2d(np.abs(ft3d)**2, fg.ugrid_raw0, fg.nu)

In [ ]:
plt.imshow(ps_2d[51:], origin='lower',norm=LogNorm())

In [ ]:
from spore.mock.convert_sim_to_vis import pos_to_baselines, baselines_to_u0

pos = np.genfromtxt("../../../clustering_counts_paper/hex_pos.txt")[:,[1,2]]  # MWA tile positions
baselines = pos_to_baselines(pos)

Now we plot the values of $(u,v)$ for the baselines, at the reference frequency of $\nu_0=150$ MHz. We also show a square corresponding to the range of $u,v$ calculated for our simulated grid.

## All-Component Mock

In [ ]:
from spore.mock.foregrounds import GalacticForegrounds, ThermalForegrounds, Foregrounds

In [ ]:
gal_sky = spatial_dist.PureClustering_FlatSky(f0 = 1., # Frequencies (as ratios to ref. freq.) to evaluate sky at.
                                              sky_size = 2.0,             # Size of sky in l,m co-ordinates
                                              ncells = 256,              # Number of grid cells along a side 
                                              seed = 4321,                # Optional seed to enforce the same realisation on every run
                                              power_spectrum = lambda u : 0.01 * (u/0.1)**-2.7,   # A function defining an isotropic power spectrum of sources as a function of scale u.
                                              use_lognormal=False
                                             )

gal_spec_index = spectral_index.UniversalDist(0.55)

gal_fg = GalacticForegrounds(fluctation_temp=253 * 0.8, Aeff=20.,
                             beam_model=gaussbeam,
                             spec_index_model = gal_spec_index,
                             spatial_dist = gal_sky
                             )

In [ ]:
th_fg = ThermalForegrounds(Aeff=20., Tsys=100, deltaT = 1000., dnu=1e5)

In [ ]:
fg = Foregrounds(beam_model=gaussbeam,
                 galactic_kwargs = dict(fluctation_temp=253*0.8,
                                        Aeff = 20.,
                                        spec_index_model=gal_spec_index,
                                        spatial_dist = gal_sky),
                 point_source_kwargs = dict(spec_index_model=spec_index_dist,
                                            spatial_dist = poisson_sky,
                                            source_counts = pl_source_counts))

In [ ]:
plt.imshow(fg.sky[0])
plt.colorbar()

In [ ]:
plt.imshow(fg.point_source_foregrounds.sky[0])
plt.colorbar()

In [ ]:
plt.imshow(fg.galactic_foregrounds.sky[0])
plt.colorbar()

In [ ]:
plt.imshow(np.abs(fg.visibility[0]))

In [ ]:
poisson_sky.seed=None
gal_sky.seed=None
th_fg.seed = None

u, cov_full = visibility_covariance(foreground_model=Foregrounds,
                                    niter=100,
                                    beam_model=gaussbeam,
                                    galactic_kwargs = dict(
                                        fluctation_temp=253*0.8,
                                        Aeff = 20.,
                                        spec_index_model=gal_spec_index,
                                        spatial_dist = gal_sky),
                                    point_source_kwargs = dict(
                                        spec_index_model=spec_index_dist,
                                        spatial_dist = poisson_sky,
                                        source_counts = pl_source_counts))

u, cov_gal = visibility_covariance(foreground_model=GalacticForegrounds,
                                    niter=100,
                                    beam_model=gaussbeam,
                                    fluctation_temp=253*0.8,
                                    Aeff = 20.,
                                    spec_index_model=gal_spec_index,
                                    spatial_dist = gal_sky)

u, cov_ps = visibility_covariance(foreground_model=PointSourceForegrounds,
                                  niter=100,
                                  beam_model=gaussbeam,
                                  spec_index_model=spec_index_dist,
                                  spatial_dist = poisson_sky,
                                  source_counts = pl_source_counts)


In [ ]:
plt.plot(u,cov_full, label="Total Noise")
plt.plot(u,cov_gal, label='Galactic')
plt.plot(u,cov_ps, label="PointSource")

plt.axhline(2*th_fg.sigma**4, label='Thermal', color="C3")

plt.xscale('log')
plt.yscale('log')
plt.legend()

### Using PyGSM to calculate the Galactic model

In [ ]:
# Ensure seed is None
poisson_sky.seed = None

ubins, cov = visibility_covariance(
    niter= 50,  # Number of iterations from which to estimate covariance
    beam_model=gaussbeam,
    source_counts=pl_source_counts,
    spec_index_model = spec_index_dist,
    spatial_dist = poisson_sky
)

In [ ]:
plt.plot(ubins, cov)
plt.xscale('log')
plt.yscale("log")